# 02 — Vicon CSV → Shoulder Width Estimation

This notebook:
- reads Vicon "Trajectories" CSV exports
- computes 3D shoulder width (distance between left and right shoulders)
- extracts median and mean shoulder width per participant
- exports an Excel summary file

Shoulder width is used later to normalize Quantity of Motion (QdM).

Author: Matys Précloux  
Project: SYNCOGEST

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd

from src_vicon_utils import read_vicon_csv, find_xyz_cols

## 1) Project paths (MODE-aware)

We support two modes:
- `test`: small dataset committed to GitHub
- `raw`: full dataset (ignored by git)

Input:
- `data/<MODE>/vicon_csv/*.csv`

Output:
- `results/<MODE>/vicon_shoulder_width_by_csv.xlsx`

In [2]:

# Project paths (test vs raw)

MODE = "raw"   # change to "raw" when running full dataset

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

# Data folder
ROOT = PROJECT_ROOT / "data" / MODE / "vicon_csv"

# Results folder
RESULTS_DIR = PROJECT_ROOT / "results" / MODE
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Mode:", MODE)
print("Project root:", PROJECT_ROOT)
print("Vicon CSV root:", ROOT)
print("ROOT exists:", ROOT.exists())
print("Results dir:", RESULTS_DIR.resolve())

Mode: raw
Project root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2
Vicon CSV root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/data/raw/vicon_csv
ROOT exists: True
Results dir: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/raw


## 2) Shoulder Width Computation

Shoulder width is computed as the 3D Euclidean distance between:
- right shoulder marker
- left shoulder marker

Per frame:

distance = sqrt((Rx-Lx)^2 + (Ry-Ly)^2 + (Rz-Lz)^2)

We report:
- median shoulder width (mm)
- mean shoulder width (mm)
- number of valid frames

In [ ]:
def shoulder_width_vicon(csv_path: Path) -> dict:
    """
    Compute shoulder width (mm) for P1 and P2 from a Vicon CSV.

    Shoulder width = 3D distance between left and right shoulder markers per frame.
    We report median and mean across frames.

    Tokens:
    - P1: epaule_D / epaule_G
    - P2: 2epaule_D / 2epaule_G
    """
    df = read_vicon_csv(csv_path)
    cols = list(df.columns)

    out = {"csv": str(csv_path), "file": csv_path.name}

    for pid, prefix in [("P1", ""), ("P2", "2")]:
        tokenR = f"{prefix}epaule_D"
        tokenL = f"{prefix}epaule_G"

        RX, RY, RZ = find_xyz_cols(cols, tokenR)
        LX, LY, LZ = find_xyz_cols(cols, tokenL)

        if None in (RX, RY, RZ, LX, LY, LZ):
            out[f"{pid}_error"] = f"missing {tokenR}/{tokenL} (X/Y/Z)"
            out[f"{pid}_found"] = str([RX, RY, RZ, LX, LY, LZ])
            continue
        # Compute frame-by-frame shoulder distance 
        sw = np.sqrt((df[RX] - df[LX])**2 + (df[RY] - df[LY])**2 + (df[RZ] - df[LZ])**2)
        sw = sw.replace([np.inf, -np.inf], np.nan).dropna()
        # Remove invalid values
        out[f"{pid}_shoulder_width_median_mm"] = float(sw.median())
        out[f"{pid}_shoulder_width_mean_mm"]   = float(sw.mean())
        out[f"{pid}_n_frames_used"]            = int(len(sw))

    return out

## 3) Batch Processing and Export

We iterate over all CSV files and export shoulder width estimates to Excel.

In [4]:
csv_files = sorted(ROOT.rglob("*.csv"))
print("CSV found:", len(csv_files))

rows = []
for f in csv_files:
    try:
        rows.append(shoulder_width_vicon(f))
    except Exception as e:
        rows.append({"csv": str(f), "file": f.name, "error": repr(e)})

df_out = pd.DataFrame(rows)

out_path = RESULTS_DIR / "vicon_shoulder_width_by_csv.xlsx"
df_out.to_excel(out_path, index=False)

print("✅ Saved:", out_path.resolve())
df_out.head()

CSV found: 36
✅ Saved: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/raw/vicon_shoulder_width_by_csv.xlsx


,csv,file,P1_shoulder_width_median_mm,P1_shoulder_width_mean_mm,P1_n_frames_used,P2_shoulder_width_median_mm,P2_shoulder_width_mean_mm,P2_n_frames_used
0,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD01.csv,281.600411,279.424604,18012,305.506314,304.438354,18012
1,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD02.csv,286.055610,279.573028,18000,305.121583,302.595334,18000
2,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD03.csv,289.523437,286.847498,18000,262.377859,260.316801,18000
3,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD04.csv,284.671404,284.695854,18000,348.361032,346.046499,18000
4,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD05.csv,263.954779,265.954606,18000,361.325059,361.476912,18000
